# Project Phase II

### Research Question(s)

A person's life expentancy is a given country can be predicted by different metrics of their country, including violent crime rates, health care, diet, economic development, genetics, and happiness level.

### Data Collection and Cleaning

Have an initial draft of your data cleaning
appendix. Document every step that takes your raw data file(s) and turns it
into the analysis-ready data set that you would submit with your final
project. Include text cells describing your data collection (downloading,
scraping, surveys, etc), and text cells describing any additional data
curation/cleaning (merging data frames, filtering, transformations of
variables, etc). Include code cells for data curation/cleaning, but not
collection. Note: You should be saving data in intermediate files at several
points through this process so that you are not starting from scratch every
time you change something. This also makes sure you have
documentation for everything that you have done for your reference, but
we do not necessarily need to see all of it in code form.

- Ryan: life expectancies, violent crime
- John: health care, diet, economic develpment
- Chase: genetics, happiness level

1. Find all data
- Each person does their assigned data set
2. Clean data sets
- Each person does their assigned data set
3. Group data sets (if needed)
- Each person does their assigned data set
4. Join data sets
- Chase

### Data Description

Have an initial draft of your data description (link in syllabus) section.
Your data description should be about your analysis-ready data.

### Data Limitations

Identify any potential problems with your dataset.

### Exploratory Data Analysis

Perform an (initial) exploratory data analysis.

1. Collinearity test
2. Multivariate regression model
3. Plots
4. Summary stats

### Questions for Reviewers

List specific questions for your peer reviewers
and project mentor to answer in giving you feedback on this phase.